# 1. Introduction

## 1.1 Background Information

Singapore is centrally located in South East Asia, serving as a hub for major airlines connecting tourists to dream destinations such as Thailand, Malaysia, Indonesia, etc. As a results, many will stop over for a few days to discover the country who is known as a culinary and shopping haven.

After having our daughter, my husband and I have not stopped travelling but our needs for accomodations have changed. For instance, we would need a kitchen to prepare home-cooked food for our baby so Airbnb has become one of our top options for finding an accomodation. Hence, this project is to help travellers with kids to identify appropriate accomodations to fully enjoy their stay in Singapore.

## 1.2 Problem Statement

There are 4000+ over accomodations in the Airbnb lisiting as of December 2020. How can we classify these accomodations into clusters and find more easily the right accomodation that corresponds to the budget and preferences of the travellers? This project will try to answer this question.

## 1.3 Target Audience

This report is dedicated to travellers who are looking for a short period of stay in Singapore. The results should give them a quick overview of which areas to stay that would satisfy their budget and preferences. The type of accomodations has been limited to "Entire place" as these is the accomodation most suitable for family but it can be also be relevant for travellers who seek for such type of accomodation.


# 2. Data
## 2.1 Data Require

- Listing of Singapore AirBnb accomodation
- Number of venues including restaurants, shopping malls and tourists attractions in the vicinity of the accomodation's neighborhood

# 2.2 Data Sources

In this project, we will be using free public available dataset. Below is the link to the dataset.

Airbnb accommodation data from Inside Airbnb: http://insideairbnb.com/get-the-data.html (Data compiled on December 2020)

Foursquare API to extract data on venues in a neighbourhood: https://developer.foursquare.com/